# Email Agent Demo

**What**: Build an end-to-end email-processing agent with the [`smolagents`](https://github.com/huggingface/smolagents) toolkit.

**Context**
- Based on the walkthrough "[AI Agent for Email Automation with smolagents](https://www.llmwatch.com/p/ai-agent-for-email-automation-with)".
- Original reference notebook: [Colab link](https://colab.research.google.com/drive/1SvveVaCZ__dxp9DYKKrDPN3PQYAmcJ78?usp=sharing).
- Enhancement here: added Slack webhook notifications.

**Environment variables to set before running**
- `HUGGINGFACEHUB_API_TOKEN`: Hub token for authentication.
- `SLACK_WEBHOOK_URL` (optional): for Slack notifications.
- Future Gmail integration: configure Google API client secrets/OAuth per your workflow.

**This notebook covers**
- Setup: install dependencies and authenticate to the [Hugging Face Hub](https://huggingface.co/docs/hub).
- Tools: calendar lookup, mock knowledge search, Slack notifications; `web_search` is a static placeholder.
- Agent: remote LLM-backed [`CodeAgent`](https://huggingface.co/docs/huggingface_hub/smolagents/index) with tuned prompt and policy templates.
- Usage: run on sample emails; can be extended to a real Gmail client via Google APIs/OAuth.

**Why smolagents**
- Validated tool schemas, prompt templates, and multi-step control with state.
- Built-in retries/error handling plus swappable remote or local backends.

Copyright 2024. Repository LICENSE applies.


In [ ]:
import os

# Local placeholders; keep real secrets out of Git (use env/.env)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""  # required
os.environ["SLACK_WEBHOOK_URL"] = ""        # optional
# Gmail integration example placeholders (enable/extend as needed):
# os.environ["GOOGLE_CLIENT_ID"] = ""
# os.environ["GOOGLE_CLIENT_SECRET"] = ""


In [16]:
!pip install smolagents

In [17]:
!pip install mistral_inference

In [18]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

In [19]:
import os
from huggingface_hub import login

# Set HUGGINGFACEHUB_API_TOKEN in your environment before running (keep tokens out of Git). You can also run `huggingface-cli login` locally.
token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
if not token:
    raise RuntimeError("Set HUGGINGFACEHUB_API_TOKEN before running this notebook.")
login(token=token, add_to_git_credential=True)


In [20]:
import os
from typing import Dict, Optional
from datetime import datetime, timedelta
import requests
from smolagents import CodeAgent, InferenceClientModel, tool

# Backwards compatibility for older notebook cells referencing the removed name
hfApiModel = InferenceClientModel  # type: ignore

In [21]:
@tool
def check_calendar(date: str, duration_minutes: int = 60, preferred_time: Optional[str] = None) -> str:
    """
    Checks calendar availability for a given date and duration.

    Args:
        date: Date in YYYY-MM-DD format
        duration_minutes: Duration of the meeting in minutes
        preferred_time: Optional preferred time in HH:MM format
    """
    # Dummy calendar with available time blocks (start_time, end_time)
    availabilities = {
        "2025-02-24": [
            ("09:00", "10:30"),  # 90 min slot
            ("11:00", "12:00"),  # 60 min slot
            ("14:00", "16:00"),  # 120 min slot
        ],
        "2025-02-25": [
            ("10:00", "11:30"),  # 90 min slot
            ("13:00", "15:00"),  # 120 min slot
        ],
    }

    if date not in availabilities:
        return f"No availability information for {date}"

    # Convert duration to timedelta for comparison
    required_duration = timedelta(minutes=duration_minutes)

    # Find suitable slots
    suitable_slots = []
    for start_time, end_time in availabilities[date]:
        start_dt = datetime.strptime(f"{date} {start_time}", "%Y-%m-%d %H:%M")
        end_dt = datetime.strptime(f"{date} {end_time}", "%Y-%m-%d %H:%M")
        slot_duration = end_dt - start_dt

        if slot_duration >= required_duration:
            suitable_slots.append((start_time, end_time))

    if not suitable_slots:
        return f"No slots available on {date} for a {duration_minutes} minute meeting"

    # If preferred time is specified, check if it fits in any slot
    if preferred_time:
        preferred_dt = datetime.strptime(f"{date} {preferred_time}", "%Y-%m-%d %H:%M")
        preferred_end = preferred_dt + required_duration

        for start_time, end_time in suitable_slots:
            slot_start = datetime.strptime(f"{date} {start_time}", "%Y-%m-%d %H:%M")
            slot_end = datetime.strptime(f"{date} {end_time}", "%Y-%m-%d %H:%M")

            if slot_start <= preferred_dt and slot_end >= preferred_end:
                return f"Available at preferred time {preferred_time} for {duration_minutes} minutes"

        return f"Preferred time {preferred_time} not available. Alternative slots: {', '.join([f'{start}-{end}' for start, end in suitable_slots])}"

    return f"Available slots for {duration_minutes} minute meeting on {date}: {', '.join([f'{start}-{end}' for start, end in suitable_slots])}"

In [22]:
@tool
def web_search(query: str) -> str:
    """
    Returns static search results for queries.

    Args:
        query: Search query string
    """
    search_database = {
        "project status": """
            - Project Dashboard: Latest Q1 metrics show 85% completion rate
            - Team velocity is 24 story points per sprint
            - Next milestone: API integration due March 15
            - Current blockers: Awaiting security audit completion
        """,
        "quarterly reports": """
            - Q4 2024 Performance: Revenue up 12% YoY
            - Market share increased to 23%
            - Customer satisfaction score: 4.5/5
            - Key growth areas: Enterprise solutions, API services
        """,
        "system maintenance": """
            - Scheduled maintenance: Every Sunday 2-4 AM EST
            - Emergency hotfix deployment process documented
            - Average downtime: 45 minutes per maintenance
            - Failover systems tested monthly
        """
    }

    # Find the most relevant results
    results = []
    for key, content in search_database.items():
        if any(word in query.lower() for word in key.split()):
            results.append(f"=== {key.title()} ===\n{content}")

    if not results:
        return "No specific information found for your query. Please refine your search terms."

    return "\n\n".join(results)

In [23]:
@tool
def notify_slack(summary: Dict) -> str:
    """Send a structured notification to Slack via an incoming webhook.

    Args:
        summary: Dictionary containing email analysis, actions taken, generated response, and next steps to report.
    """
    webhook_url = os.environ.get('SLACK_WEBHOOK_URL')
    if not webhook_url:
        raise RuntimeError('SLACK_WEBHOOK_URL environment variable is not set')

    notification = f"""
🔔 *Email Processing Summary*
From: {summary.get('sender', 'Unknown')}
Subject: {summary.get('subject', 'No subject')}

*Analysis*: {summary.get('analysis', 'No analysis provided')}
*Actions Taken*: {summary.get('actions', 'No actions taken')}
*Response*: {summary.get('response', 'No response generated')}
*Next Steps*: {summary.get('next_steps', 'No next steps defined')}
    """

    payload = {'text': notification.strip()}
    response = requests.post(webhook_url, json=payload, timeout=10)
    response.raise_for_status()
    return 'Notification sent to Slack'


In [24]:
def create_email_agent():
    """Creates and returns a CodeAgent for email processing."""
    # Uses the Hugging Face inference API (remote, not local execution)
    model = InferenceClientModel("meta-llama/Llama-3.1-8B-Instruct")

    prompt_templates = {
    "system_prompt": """You are an email processing agent that can check calendars, search information, and send notifications.

      Your job is to:
      1. Analyze the email to understand its intent and urgency
      2. Search for relevant information if needed
      3. Check calendar if it's a meeting request
      4. Send informative Slack notifications
      5. Prepare clear responses as draft emails without sending them before my review

      Important: Use date string directly from the email (YYYY-MM-DD format). Do not try to parse dates.

      Available tools:
      - check_calendar(date: str, duration_minutes: int = 60, preferred_time: Optional[str] = None)
        Returns availability for a given date/time/duration
      - web_search(query: str)
        Searches knowledge base and returns relevant information
      - notify_slack(summary: Dict)
        Sends notification with analysis and context

      Example outputs for different scenarios:

      1. For API issues:
      ```py
      # Search for API information
      api_info = web_search("api documentation")

      # Send urgent notification
      notify_slack({
          "sender": "dev@example.com",
          "subject": "API Rate Limit Issue",
          "analysis": "Urgent production issue with API limits",
          "actions": "Retrieved current API limits and solutions",
          "response": api_info,
          "next_steps": "Escalating to API team"
      })

      final_answer("Retrieved API documentation and notified team: " + api_info)
      ```

      2. For meeting requests:
      ```py
      # Use date string directly, don't parse it
      date = "2025-02-25"  # from email['date']
      time = "14:00"       # from email body
      duration = 120       # from email body

      # Check calendar
      availability = check_calendar(date=date, duration_minutes=duration, preferred_time=time)

      notify_slack({
          "sender": "manager@example.com",
          "subject": "Meeting Request",
          "analysis": "2-hour quarterly review meeting",
          "actions": f"Checked calendar: {availability}",
          "response": "Available slots shared",
          "next_steps": "Awaiting time confirmation"
      })

      final_answer(f"Calendar checked: {availability}")
      ```

      3. For maintenance coordination:
      ```py
      # Get maintenance info
      maintenance_info = web_search("system maintenance")

      date = "2025-02-24"  # from email['date']
      duration = 120       # estimated duration

      # Check team availability
      availability = check_calendar(date=date, duration_minutes=duration)

      notify_slack({
          "sender": "ops@example.com",
          "subject": "Maintenance Schedule Review",
          "analysis": "Need to coordinate maintenance window",
          "actions": "Checked schedule and team availability",
          "response": f"Maintenance info: {maintenance_info}\\nTeam availability: {availability}",
          "next_steps": "Schedule coordination needed"
      })

      final_answer("Retrieved maintenance schedule and team availability:\\n" +
                  f"Maintenance info: {maintenance_info}\\n" +
                  f"Team availability: {availability}")
      ```

      CRITICAL FORMAT REQUIREMENTS:
      - Begin every response with a single line starting with "Thoughts:" summarizing your plan.
      - Follow that line with a Python code block wrapped in <code>...</code> tags.
      - All tool calls and the final_answer(...) call must occur inside that code block.
      - Even if you only need to return a text reply, wrap it as Python (e.g., final_answer("...")) inside <code>...</code>.
      - Never produce a final response outside the <code>...</code> block.
      ```""",
    "final_answer": {
            "pre_messages": "Final answer:",
            "post_messages": ""
        },
    "planning": {
            "initial_plan": "",
            "update_plan_pre_messages": "",
            "update_plan_post_messages": ""
        },
    "managed_agent": {
            "task": "",
            "report": ""
        }
      }

    agent = CodeAgent(
        tools=[check_calendar, web_search, notify_slack],
        model=model,
        max_steps=5,
        verbosity_level=2,
        prompt_templates=prompt_templates
    )

    return agent

In [25]:
# Example emails for testing different workflows
example_emails = {
  "meeting_request": {
      "subject": "Project Review Meeting",
      "body": "Can we schedule a 90-minute meeting today at 2 PM to review Q1 project progress and discuss API integration timeline?",
      "sender": "john@example.com",
      "sender_name": "John Doe",
      "date": "2025-02-24"
  },
  "quarterly_review": {
      "subject": "Q4 2024 Review Meeting",
      "body": "Let's schedule a 2-hour meeting to go through the quarterly reports and plan for Q1 2025. What's your availability next week?",
      "sender": "manager@example.com",
      "sender_name": "Sarah Manager",
      "date": "2025-02-25"
  },
  "maintenance_notification": {
      "subject": "System Maintenance Schedule",
      "body": "Please review and confirm the upcoming maintenance schedule. We need to coordinate with the API team about potential downtime.",
      "sender": "ops@example.com",
      "sender_name": "Ops Team",
      "date": "2025-02-24"
  }
}

if __name__ == "__main__":
  agent = create_email_agent()

  # Process all example emails
  for email_type, email in example_emails.items():
      print(f"\n{'='*50}\nProcessing {email_type}\n{'='*50}")
      # Extract data explicitly in your prompt to make it clear for the agent
      prompt = f"""
Process this email and take appropriate actions:
Subject: {email['subject']}
Body: {email['body']}
Sender: {email['sender']}
Sender Name: {email['sender_name']}
Date: {email['date']}
"""
      result = agent.run(prompt)
      print(f"\nResult: {result}\n")


Processing meeting_request


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Process this email and take appropriate actions:                                                                │
│ Subject: Project Review Meeting                                                                                 │
│ Body: Can we schedule a 90-minute meeting today at 2 PM to review Q1 project progress and discuss API           │
│ integration timeline?                                                                                           │
│ Sender: john@example.com                                                                                        │
│ Sender Name: John Doe                                                                                           │
│ Date: 2025-02-24                                                                                                │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-3.1-8B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thoughts:                                                                                                          
Schedule a meeting to review Q1 project progress and discuss API integration timeline. Check calendar for available
slots and send a notification to the sender with the result.                                                       
                                                                                                                   
```python                                                                                                          
import datetime                                                                                                    
                                                                                                                   
# Extract relevant information from the email                                                                      
date = "2025-02-24"  # from email['date']                                                                          
time = "14:00"       # from email body (2 PM)                                                                      
duration = 90        # from email body                                                                             
                                                                                                                   
# Check calendar                                                                                                   
availability = check_calendar(date=date, duration_minutes=duration, preferred_time=time)                           
                                                                                                                   
# Send notification to the sender                                                                                  
notify_slack({                                                                                                     
    "sender": "john@example.com",                                                                                  
    "subject": "Meeting Request",                                                                                  
    "analysis": "Project review meeting",                                                                          
    "actions": f"Checked calendar: {availability}",                                                                
    "response": "Available slots shared",                                                                          
    "next_steps": "Awaiting time confirmation"                                                                     
})                                                                                                                 
                                                                                                                   
final_answer(f"Calendar checked: {availability}")                                                                  
```                                                                                                                
                                                                                                                   
Note: The `check_calendar` and `notify_slack` functions are assumed to be available and implemented as per the     
provided specification.                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Extract relevant information from the email                                                                    
  date = "2025-02-24"  # from email['date']                                                                        
  time = "14:00"       # from email body (2 PM)                                                                    
  duration = 90        # from email body                                                                           
                                                                                                                   
  # Check calendar                                                                                                 
  availability = check_calendar(date=date, duration_minutes=duration, preferred_time=time)                         
                                                                                                                   
  # Send notification to the sender                                                                                
  notify_slack({                                                                                                   
      "sender": "john@example.com",                                                                                
      "subject": "Meeting Request",                                                                                
      "analysis": "Project review meeting",                                                                        
      "actions": f"Checked calendar: {availability}",                                                              
      "response": "Available slots shared",                                                                        
      "next_steps": "Awaiting time confirmation"                                                                   
  })                                                                                                               
                                                                                                                   
  final_answer(f"Calendar checked: {availability}")                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for system maintenance information                                                                      
  maintenance_info = web_search("system maintenance")                                                              
                                                                                                                   
  # Get maintenance schedule date and duration                                                                     
  date = "2025-02-24"  # from email['date']                                                                        
  duration = 120       # estimated duration                                                                        
                                                                                                                   
  # Check team availability                                                                                        
  availability = check_calendar(date=date, duration_minutes=duration)                                              
                                                                                                                   
  # Send notification to Slack                                                                                     
  notify_slack({                                                                                                   
      "sender": "ops@example.com",                                                                                 
      "subject": "Maintenance Schedule Review",                                                                    
      "analysis": "Need to coordinate maintenance window",                                                         
      "actions": "Checked schedule and team availability",                                                         
      "response": f"Maintenance info: {maintenance_info}\nTeam availability: {availability}",                      
      "next_steps": "Schedule coordination needed"                                                                 
  })                                                                                                               
                                                                                                                   
  # Prepare draft email to Ops Team                                                                                
  draft_email = f"""                                                                                               
  Subject: Re: System Maintenance Schedule                                                                         
  From: dev@example.com                                                                                            
  To: ops@example.com                                                                                              
  Body: Hi Ops Team, I reviewed the maintenance schedule and team availability. I found the following              
  information:                                                                                                     
  Maintenance info: {maintenance_info}                                                                             
  Team availability: {availability}                                                                                
  Please confirm the schedule and let me know if we need to coordinate with the API team.                          
  """                                                                                                              
                                                                                                                   
  final_answer("Draft email prepared for Ops Team: \n" + draft_email)                                              
 ────────────────────────────────────────────────────────

[SLACK NOTIFICATION]:

🔔 *Email Processing Summary*
From: ops@example.com
Subject: Maintenance Schedule Review

*Analysis*:
Need to coordinate maintenance window

*Actions Taken*:
Checked schedule and team availability

*Response*:
Maintenance info: === System Maintenance ===

            - Scheduled maintenance: Every Sunday 2-4 AM EST
            - Emergency hotfix deployment process documented
            - Average downtime: 45 minutes per maintenance
            - Failover systems tested monthly
        
Team availability: Available slots for 120 minute meeting on 2025-02-24: 14:00-16:00

*Next Steps*:
Schedule coordination needed
    


Final answer: Draft email prepared for Ops Team: 

Subject: Re: System Maintenance Schedule
From: dev@example.com
To: ops@example.com
Body: Hi Ops Team, I reviewed the maintenance schedule and team availability. I found the following information:
Maintenance info: === System Maintenance ===

            - Scheduled maintenance: Every Sunday 2-4 AM EST
            - Emergency hotfix deployment process documented
            - Average downtime: 45 minutes per maintenance
            - Failover systems tested monthly
        
Team availability: Available slots for 120 minute meeting on 2025-02-24: 14:00-16:00
Please confirm the schedule and let me know if we need to coordinate with the API team.

[Step 1: Duration 1.45 seconds| Input tokens: 877 | Output tokens: 295]


Result: Draft email prepared for Ops Team: 

Subject: Re: System Maintenance Schedule
From: dev@example.com
To: ops@example.com
Body: Hi Ops Team, I reviewed the maintenance schedule and team availability. I found the following information:
Maintenance info: === System Maintenance ===

            - Scheduled maintenance: Every Sunday 2-4 AM EST
            - Emergency hotfix deployment process documented
            - Average downtime: 45 minutes per maintenance
            - Failover systems tested monthly
        
Team availability: Available slots for 120 minute meeting on 2025-02-24: 14:00-16:00
Please confirm the schedule and let me know if we need to coordinate with the API team.


